In [ ]:
### 가장 마지막 댓글부터 크롤링 하는 코드

In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from time import sleep
import requests
import re
import pandas as pd
import numpy as np
import os
import urllib

from selenium.common.exceptions import NoSuchElementException
import chromedriver_autoinstaller
import csv

import pickle

import warnings
warnings.filterwarnings('ignore')

In [2]:
review_img_path = './review_img'
os.makedirs(review_img_path, exist_ok=True)
product_img_path = './product_img'
os.makedirs(product_img_path, exist_ok=True)

In [3]:
# 제품 이미지 크롤링

page_link = "https://www.musinsa.com/categories/item/002?d_cat_cd=002&brand=&list_kind=small&sort=sale_high&sub_sort=1y&page=1&display_cnt=90&group_sale=&exclusive_yn=&sale_goods=&timesale_yn=&ex_soldout=&kids=&color=&price1=&price2=&shoeSizeOption=&tags=&campaign_id=&includeKeywords=&measure="

product_img_click_link = []


options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
# driver = webdriver.Chrome(ChromeDriverManager().install())
driver.implicitly_wait(3)
driver.get(page_link)
req = requests.get(page_link, headers={"User-Agent": "Mozilla/5.0"})

item_num = 1
page_button_num = 4

while(len(product_img_click_link)<=100):
    befor_page = driver.current_url
    # 제품 페이지 접속
    div_num = len(driver.find_elements_by_css_selector('#searchList > li:nth-child('+str(item_num)+') > div')) - 1

    try:
        button = driver.find_element_by_xpath('//*[@id="searchList"]/li['+str(item_num)+']/div['+str(div_num)+']/div[1]/a')
    except:
        try:
            button = driver.find_element_by_xpath('//*[@id="searchList"]/li['+str(item_num)+']/div['+str(div_num - 1)+']/div[1]/a')
        except:
            button = driver.find_element_by_xpath('//*[@id="searchList"]/li['+str(item_num)+']/div['+str(div_num - 2)+']/div[1]/a')
        
    driver.execute_script("arguments[0].click();", button)
    sleep(0.5)
        
    option = len(driver.find_elements_by_css_selector('#goods_opt_area > select'))
    
    print(item_num,"번째 아이템",option,"개의 옵션")
    # 제품에 옵션이 1개인 제품만 이미지 크롤링 및 웹페이지 주소 append
    if option <= 1:
        product_img_url = driver.find_element_by_xpath('//*[@id="bigimg"]').get_attribute('src')
        url = driver.current_url
        
        urllib.request.urlretrieve(product_img_url, product_img_path +'/'+str(len(product_img_click_link))+".jpg")
        
        print("추가할 url: ",url)
        product_img_click_link.append(url)
        item_num+=1
    else:
        item_num+=1  
    driver.get(befor_page)
    sleep(0.5)
        
    if item_num == 91:
        item_num = 1
        page_button = driver.find_element_by_xpath('//*[@id="goods_list"]/div[2]/div[5]/div/div/a['+str(page_button_num)+']')
        page_button_num += 1
        driver.execute_script("arguments[0].click();", page_button)
        sleep(0.5)



====== WebDriver manager ======
Current google-chrome version is 111.0.5563
Get LATEST chromedriver version for 111.0.5563 google-chrome
Driver [C:\Users\taeyoung\.wdm\drivers\chromedriver\win32\111.0.5563.64\chromedriver.exe] found in cache


1 번째 아이템 1 개의 옵션
추가할 url:  https://www.musinsa.com/app/goods/1778404
2 번째 아이템 2 개의 옵션
3 번째 아이템 2 개의 옵션
4 번째 아이템 1 개의 옵션
추가할 url:  https://www.musinsa.com/app/goods/1558197
5 번째 아이템 2 개의 옵션
6 번째 아이템 1 개의 옵션
추가할 url:  https://www.musinsa.com/app/goods/2704962
7 번째 아이템 1 개의 옵션
추가할 url:  https://www.musinsa.com/app/goods/2064545
8 번째 아이템 1 개의 옵션
추가할 url:  https://www.musinsa.com/app/goods/1173366
9 번째 아이템 1 개의 옵션
추가할 url:  https://www.musinsa.com/app/goods/1778408
10 번째 아이템 1 개의 옵션
추가할 url:  https://www.musinsa.com/app/goods/2238440
11 번째 아이템 1 개의 옵션
추가할 url:  https://www.musinsa.com/app/goods/1576700
12 번째 아이템 1 개의 옵션
추가할 url:  https://www.musinsa.com/app/goods/2064478
13 번째 아이템 1 개의 옵션
추가할 url:  https://www.musinsa.com/app/goods/1609490
14 번째 아이템 1 개의 옵션
추가할 url:  https://www.musinsa.com/app/goods/1283757
15 번째 아이템 2 개의 옵션
16 번째 아이템 1 개의 옵션
추가할 url:  https://www.musinsa.com/app/goods/1758197
17 번째 아이템 1 개의 옵션
추가할 url:  https://www.musinsa.com/app/goods/1108007
18 번째 아이템 1 개의 옵션
추가할 url: 

In [5]:
with open("product_link_list.pkl","wb") as f:
    pickle.dump(product_img_click_link, f)

In [3]:
# 리뷰 이미지 크롤링
with open("product_link_list.pkl","rb") as f:
    product_link_list = pickle.load(f)
o_count = 0
x_count = 0
for idx, page_address in enumerate(product_link_list):
    # 무신사 페이지 열기
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.implicitly_wait(3)
    driver.get(page_address)
    req = requests.get(page_address, headers={"User-Agent": "Mozilla/5.0"})
    
    # 리뷰 이미지 저장 폴더 생성
    review_img_path = './review_img/'+str(idx)+'_review_img'
    # product_img_path = './review_img/'+i['title']+'_review_img'
    os.makedirs(review_img_path, exist_ok=True)
    os.makedirs(review_img_path+'/적립O', exist_ok=True)
    os.makedirs(review_img_path+'/적립X', exist_ok=True)
    
    # 가장 마지막 페이지로 이동
    button = driver.find_element_by_xpath('//*[@id="reviewListFragment"]/div[11]/div[2]/div/a[9]')
    driver.execute_script("arguments[0].click();", button)
    sleep(2)
    
    # 마지막 페이지는 그냥 넘김
    review_num = len(driver.find_elements_by_css_selector('#reviewListFragment > div'))
    # print("마지막페이지 리뷰 개수: ", review_num)
    # button_num = len(driver.find_elements_by_css_selector('#reviewListFragment > div.nslist_bottom > div.pagination.textRight > div > a')) - 3
    # print("마지막페이지 버튼 개수:", button_num)
    # LP_button = driver.find_element_by_xpath('//*[@id="reviewListFragment"]/div['+str(review_num)+']/div[2]/div/a['+str(button_num)+']')
    LP_button = driver.find_element_by_xpath('//*[@id="reviewListFragment"]/div['+str(review_num)+']/div[2]/div/a[2]')
    driver.execute_script("arguments[0].click();", LP_button)
    sleep(2)
    
    # 마지막 페이지 바로 이전부터 반복을 통해 크롤링
    for k in range(10):
        for i in range(6,1,-1):
            for j in range(1,11):
                # 이미지 정보 가져오기
                try:
                    imgUrl = driver.find_element_by_xpath('//*[@id="reviewListFragment"]/div['+str(j)+']/div[4]/div[3]/div/ul/li/img').get_attribute('src')
                except:
                    imgUrl = driver.find_element_by_xpath('//*[@id="reviewListFragment"]/div['+str(j)+']/div[4]/div[3]/div/ul/li[1]/img').get_attribute('src')
                
                # 리뷰 댓글 정보 가져오기 및 폴더에 이미지 저장
                review_comment_id = driver.find_element_by_xpath('//*[@id="reviewListFragment"]/div['+str(j)+']/div[6]/div').get_attribute('c_idx')
                review_comment_len = len(driver.find_elements_by_css_selector('#comment_'+str(review_comment_id)+' > div'))+1
    
                print(review_comment_id)
                print(review_comment_len)
                if review_comment_len == 1:
                    urllib.request.urlretrieve(imgUrl, review_img_path + "/적립O/"+str(count)+".jpg")
                    o_count+=1
                    sleep(0.5)
                elif review_comment_len == 2:
                    try:
                        path = '//*[@id="comment_'+str(review_comment_id)+'"]/div/div[3]/p/span[2]'
                        dat = driver.find_element_by_xpath(path)
                        urllib.request.urlretrieve(imgUrl, review_img_path + "/적립X/"+str(count)+".jpg")
                        x_count+=1
                    except:
                        urllib.request.urlretrieve(imgUrl, review_img_path + "/적립O/"+str(count)+".jpg")
                        o_count+=1
                        continue
                else:
                    for m in range(1,review_comment_len):
                        try:
                            path = '//*[@id="comment_'+str(review_comment_id)+'"]/div['+str(m)+']/div[3]/p/span[2]'
                            dat = driver.find_element_by_xpath(path)
                            urllib.request.urlretrieve(imgUrl, review_img_path + "/적립X/"+str(count)+".jpg")
                            x_count+=1
                            break
                        except:
                            continue
                    urllib.request.urlretrieve(imgUrl, review_img_path + "/적립O/"+str(count)+".jpg")
                    o_count+=1
                    sleep(0.5)
                
            button = driver.find_element_by_xpath('//*[@id="reviewListFragment"]/div[11]/div[2]/div/a['+str(i)+']')
            driver.execute_script("arguments[0].click();", button)
            sleep(0.5)
    o_count = 0
    x_count = 0



====== WebDriver manager ======
Current google-chrome version is 111.0.5563
Get LATEST chromedriver version for 111.0.5563 google-chrome
Driver [C:\Users\taeyoung\.wdm\drivers\chromedriver\win32\111.0.5563.64\chromedriver.exe] found in cache


16119912
1
16112796
1
16095371
2
16087406
1
16084805
2
16082565
1
16023407
2
16021564
1
16020152
1
16016531
1
16171213
3
16161514
2
16154830
1
16144146
1
16141715
1
16128233
1
16128254
1
16127249
1
16126548
1
16122455
1
16230878
2
16229989
2
16213954
2
16209353
1
16208995
2
16190710
1
16189034
2
16187607
2
16185300
1
16175371
2
16303726
2
16295233
2
16287917
1
16280435
1
16270658
1
16269413
1
16257348
1
16252578
1
16250223
1
16234121
2
16366810
1
16348301
1
16348278
1
16347994
1
16347371
2
16346897
1
16333057
2
16313409
1
16309819
1
16306900
1
16419892
1
16418257
1
16414437
1
16412731
1
16404985
2
16399431
1
16393984
1
16377751
1
16374065
2
16367346
2
16482767
2
16482559
1
16481369
1
16481289
1
16473151
2
16470217
1
16462130
1
16453190
1
16452184
1
16421723
1
16549461
2
16548645
1
16547335
3
16523573
1
16517074
1
16501360
1
16499961
1
16498798
1
16493738
1
16489607
1
16622791
1
16621170
1
16619704
1
16617462
1
16571817
1
16594720
1
16580243
1
16576993
1
16575176
2
16566754
1


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=111.0.5563.65)
Stacktrace:
Backtrace:
	(No symbol) [0x0063DCE3]
	(No symbol) [0x005D39D1]
	(No symbol) [0x004E4DA8]
	(No symbol) [0x004CD0D3]
	(No symbol) [0x0052EA8B]
	(No symbol) [0x0053D093]
	(No symbol) [0x0052ACC6]
	(No symbol) [0x00506F68]
	(No symbol) [0x005080CD]
	GetHandleVerifier [0x008B3832+2506274]
	GetHandleVerifier [0x008E9794+2727300]
	GetHandleVerifier [0x008EE36C+2746716]
	GetHandleVerifier [0x006E6690+617600]
	(No symbol) [0x005DC712]
	(No symbol) [0x005E1FF8]
	(No symbol) [0x005E20DB]
	(No symbol) [0x005EC63B]
	BaseThreadInitThunk [0x770900F9+25]
	RtlGetAppContainerNamedObjectPath [0x771D7BBE+286]
	RtlGetAppContainerNamedObjectPath [0x771D7B8E+238]
